In [3]:
import snscrape.modules.twitter  as sntwitter
import pandas as pd
from datetime import timedelta, datetime
tweets_list=[]
keyword='elon musk'
end_date=datetime.now()
start_date=enddate-timedelta(100)
for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f"{keyword} since:{start_date.date()} until:{end_date.date()}").get_items()):
    if i >= 1000:  # Only scrape 1000 tweets
        break
    tweets_list.append([tweet.date, tweet.id, tweet.url, tweet.rawContent, tweet.user.username, tweet.replyCount, tweet.retweetCount, tweet.lang, tweet.sourceLabel, tweet.likeCount])
df=pd.DataFrame(tweets_list,columns=['date', 'id', 'url', 'tweet_content', 'user', 'reply_count', 'retweet_count', 'language', 'source', 'like_count'])


/tmp/ipykernel_9601/28299530.py:11: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets_list.append([tweet.date, tweet.id, tweet.url, tweet.content, tweet.user.username, tweet.replyCount, tweet.retweetCount, tweet.lang, tweet.sourceLabel, tweet.likeCount])


In [4]:
df.head()

,date,id,url,tweet_content,user,reply_count,retweet_count,language,source,like_count
0,2023-04-07 23:59:59+00:00,1644490234145435649,https://twitter.com/Roxxlive/status/1644490234...,@elonmusk @andst7 i mean they drown anyway,Roxxlive,0,0,en,Twitter Web App,0
1,2023-04-07 23:59:59+00:00,1644490231641694208,https://twitter.com/EFFETTI/status/16444902316...,@elonmusk @maccaw ^ @ 2AM = *whAtevER ?!!?.. ....,EFFETTI,0,0,en,Twitter for iPhone,0
2,2023-04-07 23:59:58+00:00,1644490228076548097,https://twitter.com/RobsTaylor97/status/164449...,@elonmusk @SawyerMerritt @WholeMarsBlog Ross G...,RobsTaylor97,0,0,en,Twitter for Android,0
3,2023-04-07 23:59:57+00:00,1644490225413132288,https://twitter.com/Lisaautry0101/status/16444...,@BillyM2k @cb_doge @elonmusk What’s your EQ?,Lisaautry0101,0,0,en,Twitter for iPhone,0
4,2023-04-07 23:59:56+00:00,1644490222871228416,https://twitter.com/Michael981981/status/16444...,@elonmusk \nThis your word about Zombie Apocal...,Michael981981,0,0,en,Twitter for Android,0


In [6]:
import pymongo
client=pymongo.MongoClient("mongodb://localhost:27017/")
db=client.twitter_db
scraped_data={"Scraped Word":keyword,
             "Scraped Date":str(end_date.date()),
              "Scraped Data":tweets_list
             }
db.twitter_data.insert_one(scraped_data)

In [ ]:
import streamlit as st
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime, timedelta
# import datetime
import pymongo
import base64

client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["twitter_db"]
collection = db["twitter_data"]

# Define a function to scrape Twitter data
def scrape_twitter_data(keyword, start_date, end_date, tweet_count):
    tweets_list = []
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(f"{keyword} since:{start_date} until:{end_date}").get_items()):
        if i >= tweet_count:
            break
        tweets_list.append([tweet.date, tweet.id, tweet.url, tweet.content, tweet.user.username, tweet.replyCount, tweet.retweetCount, tweet.lang, tweet.sourceLabel, tweet.likeCount])
    df = pd.DataFrame(tweets_list, columns=['date', 'id', 'url', 'tweet_content', 'user', 'reply_count', 'retweet_count', 'language', 'source', 'like_count'])
    return df

def  app():
    {
     st.title("Twitter Datta Scraper and Uploader")
     keyword=st.sidebar.text_input("Enter the Keywors or  Hashtag to search") 
     start_date=st.sidebar.time_input("SElect the start date")   
     end_date=st.sidebar.time_input("Enter the end date") 
     tweet_count=st.sidebar.number_input("Enter the number to scrape",min=1,max=1000,val=100)
     
     show_data=st.checkbox("Show scraped data")
     upload_data=st.button("Upload data to MongoDB")
     download_csv=st.button("Download data to csv")
     download_json=st.button("Download data  as JSON")
        
        if  keyword and start_date and end_date  and tweet_count:
            df=scrape_twitter_data(keyword,start_date,end_date,tweet_count)
            if show_data:
                st.write(df)
        if upload_data and 'df' in locals():
              scraped_data = {
            "Scraped Word": keyword,
            "Scraped Start Date": str(start_date),
            "Scraped End Date": str(end_date),
            "Scraped Data": df.to_dict('records')
        }
        collection.insert_one(scraped_data)
        st.write("Data uploaded to MongoDB!")
        if download_csv and 'df' in locals():
        csv = df.to_csv(index=False)
        b64 = base64.b64encode(csv.encode()).decode()
        st.markdown(f'<a href="data:file/csv;base64,{b64}" download="twitter_data.csv">Download CSV File</a>', unsafe_allow_html=True)
    
    # Download the scraped data as JSON if requested
    if download_json and 'df' in locals():
        json = df.to_json(orient='records')
        b64 = base64.b64encode(json.encode()).decode()
        st.markdown(f'<a href="data:file/json;base64,{b64}" download="twitter_data.json">Download JSON File</a>', unsafe_allow_html=True)
def main:
    app()
        
if __name__='__main__':
        main()
        
        
    }